In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from game import Game
from player import Player
import pprint as pp

In [2]:
from hybiak import Hybiak
from simpleplayer import SimplePlayer
from golab_uladzislau import GolabUladzislau

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
pp.pprint(getDeck())

[(9, 0),
 (10, 0),
 (11, 0),
 (12, 0),
 (13, 0),
 (14, 0),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (9, 2),
 (10, 2),
 (11, 2),
 (12, 2),
 (13, 2),
 (14, 2),
 (9, 3),
 (10, 3),
 (11, 3),
 (12, 3),
 (13, 3),
 (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(12, 1), (14, 0), (11, 2), (14, 3), (10, 3), (14, 2), (9, 1), (10, 2)]
[(11, 1), (9, 0), (13, 1), (11, 3), (13, 0), (12, 3), (13, 3), (12, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Hybiak", "Hybiak", Hybiak],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 Hybiak
==== 1 Hybiak


In [9]:
pp.pprint(full_results)

[[None,
  [[266, 734],
   [0, 0],
   [13.521, 13.75],
   [9.2, 2.756],
   4.044,
   [6.947, 1.719],
   [0.173, 0.014],
   [7.924, 6.251],
   [0, 0],
   0,
   0]],
 [None, None]]


Simple stats

In [10]:
def print_matrix(full_results, stat):
    stat_name = STAT_NAMES[stat]
    print(f"Statystyka: {stat_name}")

    strategies = [str(s[1]) for s in strategy]
    print("Strategie: ", ' '.join(strategies))

    for A in range(len(strategy)):
        current_strategy = strategy[A][1]
        results = []
        for B in range(len(strategy)):
            if A == B:
                results.append("-")
            elif A < B:
                results.append(str(full_results[A][B][stat][0]))
            else:
                results.append(str(full_results[B][A][stat][1]))
        print(f"{A} {current_strategy}: {' '.join(results)}")

for a in ANALYZE_STATS:
    print_matrix(full_results, a)

Statystyka: Wins
Strategie:  RANDOM Hybiak
0 RANDOM: - 266
1 Hybiak: 734 -
Statystyka: Draws
Strategie:  RANDOM Hybiak
0 RANDOM: - 0
1 Hybiak: 0 -
Statystyka: Moves
Strategie:  RANDOM Hybiak
0 RANDOM: - 13.521
1 Hybiak: 13.75 -
Statystyka: Cards
Strategie:  RANDOM Hybiak
0 RANDOM: - 9.2
1 Hybiak: 2.756 -
Statystyka: Checks
Strategie:  RANDOM Hybiak
0 RANDOM: - 6.947
1 Hybiak: 1.719 -
Statystyka: Draw Decisions
Strategie:  RANDOM Hybiak
0 RANDOM: - 0.173
1 Hybiak: 0.014 -
Statystyka: Cheats
Strategie:  RANDOM Hybiak
0 RANDOM: - 7.924
1 Hybiak: 6.251 -
Statystyka: Errors
Strategie:  RANDOM Hybiak
0 RANDOM: - 0
1 Hybiak: 0 -


In [11]:
# Definicja strategii
strategy = [
    ["GolabUladzislau", "GolabUladzislau", GolabUladzislau],
    ["Hybiak", "Hybiak", Hybiak],
]

# Inicjalizacja wyników
full_results = [[None for _ in strategy] for _ in strategy]

# Porównanie graczy
for A, (name_A, _, player_A) in enumerate(strategy[:-1]):
    for B, (name_B, _, player_B) in enumerate(strategy[A+1:], start=A+1):
        print(f"Porównanie {name_A} z {name_B}")
        results = comparePlayers(player_A, player_B)
        full_results[A][B] = results

import pandas as pd

def print_matrix(full_results, stat):
    stat_name = STAT_NAMES[stat]
    print(f"Statystyka: {stat_name}")

    strategies = [name for name, _, _ in strategy]

    data = []
    for A, (name_A, _, _) in enumerate(strategy):
        row = []
        for B, _ in enumerate(strategy):
            if A == B:
                row.append("-")
            elif A < B:
                row.append(str(full_results[A][B][stat][0]))
            else:
                row.append(str(full_results[B][A][stat][1]))
        data.append(row)

    df = pd.DataFrame(data, columns=strategies, index=strategies)
    print(df)

for a in ANALYZE_STATS:
    print_matrix(full_results, a)
    print()

Porównanie GolabUladzislau z Hybiak
Statystyka: Wins
                GolabUladzislau Hybiak
GolabUladzislau               -    512
Hybiak                      488      -

Statystyka: Draws
                GolabUladzislau Hybiak
GolabUladzislau               -      0
Hybiak                        0      -

Statystyka: Moves
                GolabUladzislau Hybiak
GolabUladzislau               -  7.605
Hybiak                    7.581      -

Statystyka: Cards
                GolabUladzislau Hybiak
GolabUladzislau               -  1.357
Hybiak                    1.462      -

Statystyka: Checks
                GolabUladzislau Hybiak
GolabUladzislau               -  0.337
Hybiak                    0.298      -

Statystyka: Draw Decisions
                GolabUladzislau Hybiak
GolabUladzislau               -  0.017
Hybiak                    0.009      -

Statystyka: Cheats
                GolabUladzislau Hybiak
GolabUladzislau               -  4.041
Hybiak                    4.137      -

St